In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, classification_report


In [5]:
df = pd.read_csv('data\\Data sheet - Sheet1.csv')
df.drop('Date', axis=1, inplace=True)

df['Roof'] = df['Roof'].fillna('Asbestos')
df['Pillars'] = df['Pillars'].fillna('Concrete')
df['Walls'] = df['Walls'].fillna('Block')
df['Damage'] = df['Damage'].fillna('None')
df['Flood Height'] = df['Flood Height'].fillna(0)



In [6]:
repair_bins = [-float('inf'), 20000, 40000, 60000, 80000, 100000, float('inf')]
value_bins = [0, 2000000, 4000000, 6000000, 8000000, 10000000, 12000000, 14000000, 16000000, 18000000, 20000000, float('inf')]
height_bins = [0, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180, 195, 210, 225, 240, 255, 270, 285, 300, 315, 330, 345, 360, 375, 390, 405, 420, 435,float('inf')]

df['Estimated Repair value bins'] = pd.cut(df['Estimated Repair value'], bins=repair_bins, labels=False)
df['Estimated value bins'] = pd.cut(df['Estimated value'], bins=value_bins, labels=False)
df['Flood Height bins'] = pd.cut(df['Flood Height'], bins=height_bins, labels=False)

In [23]:
# num_bins = 5

# df['Estimated Repair value bins'] = pd.cut(df['Estimated Repair value'], bins=num_bins, labels=False)
# df['Estimated value bins'] = pd.cut(df['Estimated value'], bins=num_bins, labels=False)

In [7]:
X_labels = ['Building Age', 'Estimated value bins', 'Floors', 'Building Height', 'Walls', 'Roof', 'Pillars', 'Flood Height']
y_label = ['Estimated Repair value bins']
categorical_cols = ['Walls', 'Roof', 'Pillars']

X = df[X_labels]
y = df[y_label]

In [8]:
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(df[categorical_cols])

encoded_cols = encoder.get_feature_names_out(categorical_cols)
X_encoded_df = pd.DataFrame(X_encoded.toarray(), columns=encoded_cols)

X_numerical = X.drop(columns=categorical_cols)
X_final = pd.concat([X_encoded_df, X_numerical], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)


In [32]:
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100,random_state=42),
    'Linear Regression': LinearRegression(),
    'Decision Tree Regression': DecisionTreeRegressor(random_state=42),
    'Gradient Boosting Regression': GradientBoostingRegressor(random_state=42),
    'K-Nearest Neighbors Regression': KNeighborsRegressor()
}

In [ ]:
for model_name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"{model_name} MSE: {mse}")

In [9]:
models = {
    'Random Forest Classifier': RandomForestClassifier(random_state=42),
    'Support Vector Classifier': SVC(kernel='linear', random_state=42),
    'K-Nearest Neighbors Classifier': KNeighborsClassifier()
}
classification_reports = {}

In [10]:
for model_name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)
    classification_reports[model_name] = report
    print(f"{model_name} accuracy: {accuracy}")

e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precisio

Random Forest Classifier accuracy: 0.6
Support Vector Classifier accuracy: 0.9
K-Nearest Neighbors Classifier accuracy: 0.5


e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\ACA\S7\code\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision i

In [35]:
print(classification_reports['Support Vector Classifier'])
print(classification_reports['Random Forest Classifier'])
print(classification_reports['K-Nearest Neighbors Classifier'])

              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       1.00      1.00      1.00         2
           3       1.00      0.67      0.80         3

    accuracy                           0.90        10
   macro avg       0.94      0.89      0.90        10
weighted avg       0.92      0.90      0.89        10

              precision    recall  f1-score   support

           0       0.62      1.00      0.77         5
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         0
           3       1.00      0.33      0.50         3

    accuracy                           0.60        10
   macro avg       0.41      0.33      0.32        10
weighted avg       0.61      0.60      0.53        10

              precision    recall  f1-score   support

           0       0.62      1.00      0.77         5
           1       0.00      0.00      0.00         2
           2       0.

In [41]:
joblib.dump(models['Support Vector Classifier'], 'svm_model.joblib')

['svm_model.joblib']

In [42]:
joblib.dump(encoder, 'encoder.joblib')


['encoder.joblib']

In [51]:
data = X.head(1)

In [47]:
model1 = joblib.load('svm_model.joblib')
encoder1 = joblib.load('encoder.joblib')

In [52]:
encoded_data = encoder1.transform(data[categorical_cols])
encoded_cols = encoder1.get_feature_names_out(categorical_cols)
encoded_data_df = pd.DataFrame(encoded_data.toarray(), columns=encoded_cols)

data_numerical = data.drop(columns=categorical_cols)
data_final = pd.concat([encoded_data_df, data_numerical], axis=1)

print(model1.predict(data_final))

[5]
